<a href="https://colab.research.google.com/github/linked0/deep-learning/blob/master/AAMY/Sentiment_Classification_Solutions_my.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Classification & How To "Frame Problems" for a Neural Network

### What You Should Already Know
* neural networks, forward and back-propagation
* stochastic gradient descent
* mean squared error
* and train/test spits

### Where to Get Help if You Need It
* Re-watch previous Udacity Lectures
* Levderage the recommended Course Reading Materail - Grokking Deep Learning
* Shoot me a tweet @iamtrask

### Tutorial Outline:
* Intro: The Importance of "Framing a Problem" (this lesson)
....


# 준비

In [7]:
!apt-get install -y -qq software-properties-common module-init-tools > /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse > /dev/nul

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
###### git partially cloning
repo='deep-learning'
sub_dir='sentiment-network'
url='https://github.com/linked0/' + repo + '.git'
!mkdir $repo
%cd $repo
!git init
!git remote add -f origin $url
!git config core.sparsecheckout true
!echo $sub_dir >> .git/info/sparse-checkout
!git pull --recurse-submodules origin master
%cd $sub_dir
!ls

/content/deep-learning
Initialized empty Git repository in /content/deep-learning/.git/
Updating origin
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1387 (delta 47), reused 19 (delta 11), pack-reused 1307
Receiving objects: 100% (1387/1387), 127.04 MiB | 27.89 MiB/s, done.
Resolving deltas: 100% (421/421), done.
From https://github.com/linked0/deep-learning
 * [new branch]      master     -> origin/master
From https://github.com/linked0/deep-learning
 * branch            master     -> FETCH_HEAD
/content/deep-learning/sentiment-network
labels.txt				     sentiment_network_2.png
requirements.txt			     sentiment_network.png
reviews.txt				     sentiment_network_pos.png
Sentiment_Classification_Projects.ipynb      sentiment_network_sparse_2.png
Sentiment_Classification_Solutions.ipynb     sentiment_network_sparse.png
Sentiment_Classification_Solutions_my.ipynb


labels.txt*                                  sentiment_network_2.png*
requirements.txt*                            sentiment_network.png*
reviews.txt*                                 sentiment_network_pos.png*
Sentiment_Classification_Projects.ipynb*     sentiment_network_sparse_2.png*
Sentiment_Classification_Solutions.ipynb*    sentiment_network_sparse.png*
Sentiment_Classification_Solutions_my.ipynb


In [0]:
from google.colab import files
files.download("reviews.txt")
 

# Lesson: Curate a Dataset

In [0]:
def pretty_print_review_and_label(i):
  print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt', 'r') # What we know!
review = list(map(lambda x:x[:-1], g.readlines()))
g.close()

g = open('labels.txt', 'r')
labels = list(map(lambda x:x[:-1].upper(), g.readlines()))
g.close()

In [12]:
len(review)

25000

In [13]:
review[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [14]:
labels[0]

'POSITIVE'